In [1]:
import requests
import folium
import pandas as pd
from sklearn.cluster import KMeans
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import json
import requests
from bs4 import BeautifulSoup as bSoup
import numpy as np
import re
from geopy.geocoders import Nominatim as geocoder
import csv
from pandas import json_normalize
import matplotlib.pyplot as mp
import seaborn as sns

print("Library import")
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text
soup = bSoup(data,"html5lib")

contents = []
table = soup.find('table')
for row in table.findAll('td'):
    col = {}
    #Skip cells of table with not assinged.
    if row.span.text == 'Not assigned':
        pass
    #Split the contents of table cell, where first 3 chars are the postal code, and borough is contained outside the (),
    #and the neighbourhood is within the ().
    else:
        col['PCode'] = row.p.text[:3]
        col['Borough'] = (row.span.text).split('(')[0]
        col['Neighbourhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        contents.append(col)
tblCont = pd.DataFrame(contents)
tblCont['Borough'] = tblCont['Borough'].replace({'EtobicokeNorthwest':'Etobicoke Northwest',
                                            'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                            'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                            'MississaugaCanada Post Gateway Processing Centre':'Mississauga',
                                            'East YorkEast Toronto':'East York/East Toronto'})
print("Successful scrap")


Library import
Successful scrap


#### Csv file from the url contains the long and lat of Toronto, just have to parse out the ones that match the scraped postal codes.


In [2]:
longLatCSV = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
parsePCodes = pd.read_csv(longLatCSV)

#### Join the parsePCodes with tblCont on postal codes, and remove added columns from the dataframe that have NaN data, delete duplicate data, and reset the index.

In [3]:
finalizedLatLong = pd.merge(tblCont, parsePCodes, left_on = 'PCode', right_on = 'Postal Code')
finalizedLatLong = finalizedLatLong.drop('Postal Code', axis = 1)
finalizedLatLong = finalizedLatLong.drop_duplicates()
finalizedLatLong.index = range(len(finalizedLatLong))
finalizedLatLong.to_csv('finalizedLatLong.csv')
finalizedLatLong.head()

,PCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


#### Foursquare api info

In [4]:
LIMIT = 100
radius = 500
VERSION = 20220410
CLIENT_ID = 'LUQWIMZ52YZWINDV0F1C01R40VFQM2QGE3NV54U4X510FRXC'
CLIENT_SECRET = '35NN2UF2WG33WGN20XPJN20MWA44ZE3VFBNETVU55XCMEMEA'
couProvCity = 'Toronto, Ontario, Canada'

#### Use foursquare agent

In [5]:
gLoc = geocoder(user_agent = "AI-Proj-Key-2022")
cityLoc = gLoc.geocode(couProvCity)
latitude = cityLoc.latitude
longitude = cityLoc.longitude
tlongitude = longitude
tlatitude = latitude

mapOfRegion = folium.Map(location = [latitude, longitude], zoom_start = 11)
#mapOfRegion

#### Place markers on neighbourhoods that have been scraped.

In [6]:
for lati, longi, borough, neighbourhood in zip(finalizedLatLong['Latitude'], finalizedLatLong['Longitude'],finalizedLatLong['Borough'],finalizedLatLong['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lati, longi],
    fill = True,
    fill_opacity = 0.1,
    color = '#cc8131',
    fill_color = 'yellow',
    radius = 6,
    popup = label,
    parse_html = False).add_to(mapOfRegion)
mapOfRegion

#### Reading the needed csv files to datasets to join on common columns.

In [7]:
dataCrime = pd.read_csv ('Neighbourhood_Crime_Rates_(Boundary_File)_.csv')
dataNeighProfiles = pd.read_csv ('neighbourhood-profiles-2016-csv.csv')
dataNeigh = pd.read_csv ('Neighbourhoods-4326.csv')
#print(dataNeighProfiles)

#### Joining dataNeigh with dataCrime on the same Neighbourhood name and AREA_DESC name.

In [8]:
crimeAndNeigh = dataCrime.merge(dataNeigh,left_on = 'Neighbourhood', right_on = 'AREA_DESC')
crimeAndNeigh.to_csv('crimeAndNeigh.csv')
crimeAndNeigh = pd.merge(crimeAndNeigh, finalizedLatLong, on = 'Neighbourhood')

#crimeAndNeigh.head()
#print(crimeAndNeigh)

#### Merging crimeAndNeigh, and datatNeighProfiles together.

In [9]:
print(crimeAndNeigh)
totalDF = pd.merge(crimeAndNeigh, dataNeighProfiles, on = 'Neighbourhood')
totalDF.to_csv('totalDF2.csv')

    OBJECTID_x        Neighbourhood  Hood_ID  Population  Assault_2014  \
0            7          The Beaches       63       21567            83   
1            9     Thorncliffe Park       55       21108            86   
2           11   Humewood-Cedarvale      106       14365            43   
3           15  Scarborough Village      139       16724           161   
4           20        Humber Summit       21       12416            76   
5           39     Victoria Village       43       17510           118   
6           62    Hillcrest Village       48       16934            63   
7           97      Willowdale West       37       16936            86   
8          112               Woburn      137       53485           352   
9          114      Bayview Village       52       21396            79   
10         119               Weston      113       17992           180   

    Assault_2015  Assault_2016  Assault_2017  Assault_2018  Assault_2019  \
0            108            86     

#### Print out columns to find correlation criteria.

In [10]:
features1 = list(["BreakandEnter_2015","Divorced","$30,000 to $39,999"])
print(totalDF.columns.tolist())
#totalDF[features1].corr()

['OBJECTID_x', 'Neighbourhood', 'Hood_ID', 'Population', 'Assault_2014', 'Assault_2015', 'Assault_2016', 'Assault_2017', 'Assault_2018', 'Assault_2019', 'Assault_AVG', 'Assault_CHG', 'Assault_Rate_2019', 'AutoTheft_2014', 'AutoTheft_2015', 'AutoTheft_2016', 'AutoTheft_2017', 'AutoTheft_2018', 'AutoTheft_2019', 'AutoTheft_AVG', 'AutoTheft_CHG', 'AutoTheft_Rate_2019', 'BreakandEnter_2014', 'BreakandEnter_2015', 'BreakandEnter_2016', 'BreakandEnter_2017', 'BreakandEnter_2018', 'BreakandEnter_2019', 'BreakandEnter_AVG', 'BreakandEnter_CHG', 'BreakandEnter_Rate_2019', 'Homicide_2014', 'Homicide_2015', 'Homicide_2016', 'Homicide_2017', 'Homicide_2018', 'Homicide_2019', 'Homicide_AVG', 'Homicide_CHG', 'Homicide_Rate_2019', 'Robbery_2014', 'Robbery_2015', 'Robbery_2016', 'Robbery_2017', 'Robbery_2018', 'Robbery_2019', 'Robbery_AVG', 'Robbery_CHG', 'Robbery_Rate_2019', 'TheftOver_2014', 'TheftOver_2015', 'TheftOver_2016', 'TheftOver_2017', 'TheftOver_2018', 'TheftOver_2019', 'TheftOver_AVG', 'T

#### Getting correlation between B & E 2015, income of 30k - 39k, and Male 20-24 yrs old

In [11]:
totalDF["    $30,000 to $39,999"] = totalDF["    $30,000 to $39,999"].str.replace(',','')
totalDF["Male: 20 to 24 years"] = pd.to_numeric(totalDF["Male: 20 to 24 years"])
totalDF["    $30,000 to $39,999"] = pd.to_numeric(totalDF["    $30,000 to $39,999"])
correlation1 = totalDF[["BreakandEnter_2015", "    $30,000 to $39,999", "Male: 20 to 24 years"]].corr()
correlation1


,BreakandEnter_2015,"$30,000 to $39,999",Male: 20 to 24 years
BreakandEnter_2015,1.000000,0.962412,0.908144
"$30,000 to $39,999",0.962412,1.000000,0.976439
Male: 20 to 24 years,0.908144,0.976439,1.000000


#### Get correlation between low inscome status 2015, 18 to 64 years (%).1, and postsecondary certificate, diploma or degree.

In [12]:
totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"] = totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"].str.replace(',','')
totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"] = pd.to_numeric(totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"])
totalDF["  18 to 64 years (%).1"] = pd.to_numeric(totalDF["  18 to 64 years (%).1"])
totalDF["  Postsecondary certificate, diploma or degree"] = pd.to_numeric(totalDF["  Postsecondary certificate, diploma or degree"])
#Below is removes commas from the column.
#totalDF[" Jamaican"] = totalDF[" Jamaican"].str.replace(',','')
correlation2 = totalDF[["  Postsecondary certificate, diploma or degree", "Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data", "  18 to 64 years (%).1"]].corr()
correlation2

,"Postsecondary certificate, diploma or degree",Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data,18 to 64 years (%).1
"Postsecondary certificate, diploma or degree",1.000000,0.886296,-0.068827
Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data,0.886296,1.000000,0.105873
18 to 64 years (%).1,-0.068827,0.105873,1.000000


In [12]:
totalDF["    Migrants"] = totalDF["    Migrants"].str.replace(',','')
totalDF["  English only"] = totalDF["  English only"].str.replace(',','')
totalDF["            Italian"] = totalDF["            Italian"].str.replace(',','')
totalDF["      Social assistance benefits: Population with an amount"] = totalDF["      Social assistance benefits: Population with an amount"].str.replace(',','')
totalDF["Population density per square kilometre"] = totalDF["Population density per square kilometre"].str.replace(',','')
totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"] = totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"].str.replace(',','')
totalDF["            Italian"] = pd.to_numeric(totalDF["            Italian"])
totalDF["    Divorced"] = pd.to_numeric(totalDF["    Divorced"])
totalDF["    Migrants"] = pd.to_numeric(totalDF["    Migrants"])
totalDF["Assault_2015"] = pd.to_numeric(totalDF["Assault_2015"])
totalDF["  English only"] = pd.to_numeric(totalDF["  English only"])
totalDF["  No certificate, diploma or degree"] = pd.to_numeric(totalDF["  No certificate, diploma or degree"])
totalDF["  Secondary (high) school diploma or equivalency certificate.1"] = pd.to_numeric(totalDF["  Secondary (high) school diploma or equivalency certificate.1"])
totalDF["    University certificate, diploma or degree at bachelor level or above.1"] = pd.to_numeric(totalDF["    University certificate, diploma or degree at bachelor level or above.1"])
totalDF["      Social assistance benefits: Population with an amount"] = pd.to_numeric(totalDF["      Social assistance benefits: Population with an amount"])
totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"] = pd.to_numeric(totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"])
totalDF["Population density per square kilometre"] = pd.to_numeric(totalDF["Population density per square kilometre"])
totalDF["Male: 30 to 34 years"] = pd.to_numeric(totalDF["Male: 30 to 34 years"])
totalDF["Female: 30 to 34 years"] = pd.to_numeric(totalDF["Female: 20 to 24 years"])
correlation3 = totalDF[["Male: 30 to 34 years", "Female: 30 to 34 years","Assault_2015","    Migrants", "Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data", "Population density per square kilometre", "      Social assistance benefits: Population with an amount", "    Divorced", "    University certificate, diploma or degree at bachelor level or above.1", "  Secondary (high) school diploma or equivalency certificate.1", "  No certificate, diploma or degree", "  English only", "            Italian"]].corr()
correlation3


,Male: 30 to 34 years,Female: 30 to 34 years,Assault_2015,Migrants,Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data,Population density per square kilometre,Social assistance benefits: Population with an amount,Divorced,"University certificate, diploma or degree at bachelor level or above.1",Secondary (high) school diploma or equivalency certificate.1,"No certificate, diploma or degree",English only,Italian
Male: 30 to 34 years,1.000000,0.927469,0.808859,0.920795,0.913016,0.102271,0.602527,0.804926,0.744406,0.815296,0.674959,0.913644,-0.203535
Female: 30 to 34 years,0.927469,1.000000,0.920660,0.784662,0.969452,-0.080710,0.740966,0.845452,0.602392,0.926397,0.838957,0.974491,-0.099205
Assault_2015,0.808859,0.920660,1.000000,0.649091,0.923946,-0.066128,0.893551,0.898822,0.374289,0.977871,0.925434,0.937246,-0.021787
Migrants,0.920795,0.784662,0.649091,1.000000,0.781527,0.084359,0.452802,0.630913,0.734873,0.644785,0.512501,0.779354,-0.341724
Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data,0.913016,0.969452,0.923946,0.781527,1.000000,-0.045753,0.762270,0.896073,0.637813,0.946471,0.847823,0.998829,-0.164482
Population density per square kilometre,0.102271,-0.080710,-0.066128,0.084359,-0.045753,1.000000,0.070212,0.130722,0.130590,-0.071194,-0.200392,-0.046007,-0.494200
Social assistance benefits: Population with an amount,0.602527,0.740966,0.893551,0.452802,0.762270,0.070212,1.000000,0.759514,0.043303,0.909286,0.920798,0.786357,-0.033588
Divorced,0.804926,0.845452,0.898822,0.630913,0.896073,0.130722,0.759514,1.000000,0.554117,0.878546,0.750766,0.895430,-0.233640
"University certificate, diploma or degree at bachelor level or above.1",0.744406,0.602392,0.374289,0.734873,0.637813,0.130590,0.043303,0.554117,1.000000,0.366280,0.140388,0.608248,-0.481965
Secondary (high) school diploma or equivalency certificate.1,0.815296,0.926397,0.977871,0.644785,0.946471,-0.071194,0.909286,0.878546,0.366280,1.000000,0.963860,0.958320,0.018089


#### Getting foursquare venue information

In [25]:
LIMIT = 50
radius = 500
category = 12094

def vensClose(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        urlDF = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories=12094&limit=50".format( 
            lat,
            lng,
            radius)
            
        headers = {"Accept": "application/json", "Authorization": "fsq3f6rpL1EABnqQ6Ie0wxtij4eZs5OX+vMZtdncijsSbNM="}
        response = requests.request("GET", urlDF, headers = headers)
        results = response.json()
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    return (venues_list)



In [27]:
lat = 43.73
lng = -79.4

url = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories{}&limit={}".format( 
            lat,
            lng,
            radius,
            category,
            LIMIT)

headers = {"Accept": "application/json", "Authorization": "fsq3f6rpL1EABnqQ6Ie0wxtij4eZs5OX+vMZtdncijsSbNM="}
response = requests.request("GET", url, headers = headers)
res = response.json()
res



{'results': [{'fsq_id': '5468ecf9498efb61639f4b8b',
   'categories': [{'id': 13033,
     'name': 'Bubble Tea Shop',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bubble_',
      'suffix': '.png'}},
    {'id': 13034,
     'name': 'Café',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
      'suffix': '.png'}},
    {'id': 13036,
     'name': 'Tea Room',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/tearoom_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 302,
   'geocodes': {'main': {'latitude': 43.728716, 'longitude': -79.403322},
    'roof': {'latitude': 43.728716, 'longitude': -79.403322}},
   'link': '/v3/places/5468ecf9498efb61639f4b8b',
   'location': {'address': '3216 Yonge St',
    'country': 'CA',
    'cross_street': 'Yonge and Lawrence',
    'formatted_address': '3216 Yonge St (Yonge and Lawrence), Toronto ON M4N 2L2',
    'locality': 'Toronto',
    'neighborhood': ['Central Toronto'],
    '

In [28]:
print("Neighbourhoods:\n")
nearVenues = vensClose(names = totalDF['Neighbourhood'],
                                 latitudes = totalDF['Latitude'],
                                 longitudes = totalDF['Longitude'])

Neighbourhoods:

The Beaches


TypeError: string indices must be integers

In [291]:
#The Beaches
lat1 = 43.6763574
lng1 = -79.2930312

url1 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat1,
            lng1,
            radius,
            category,
            LIMIT)
            
headers = {"Accept": "application/json", "Authorization": "fsq3f6rpL1EABnqQ6Ie0wxtij4eZs5OX+vMZtdncijsSbNM="}
response1 = requests.request("GET", url1, headers = headers)
res1 = response1.json()

nearVens1 = pd.DataFrame(columns = ['Neighbourhood','Neighbourhood Latitude','Neighbourhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category'])


#Thorncliffe Park
lat2 = 43.7053689
lng2 = -79.3493719

url2 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat2,
            lng2,
            radius,
            category,
            LIMIT)
            
response2 = requests.request("GET", url2, headers = headers)
res2 = response2.json()

#Humewood-Cedarvale
lat3 = 43.6937813
lng3 = -79.4281914

url3 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat3,
            lng3,
            radius,
            category,
            LIMIT)
            
response3 = requests.request("GET", url3, headers = headers)
res3 = response3.json()

#Scarborough Village
lat4 = 43.7447342
lng4 = -79.2394761

url4 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat4,
            lng4,
            radius,
            category,
            LIMIT)
            
response4 = requests.request("GET", url4, headers = headers)
res4 = response4.json()

#Humber Summit
lat5 = 43.7563033
lng5 = -79.5659633

url5 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat5,
            lng5,
            radius,
            category,
            LIMIT)

response5 = requests.request("GET", url5, headers = headers)
res5 = response5.json()

#Victoria Village
lat6 = 43.7258823
lng6 = -79.3155716

url6 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat6,
            lng6,
            radius,
            category,
            LIMIT)
            
response6 = requests.request("GET", url6, headers = headers)
res6 = response6.json()

#Hillcrest Village
lat7 = 43.8037622
lng7 = -79.3634517

url7 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat7,
            lng7,
            radius,
            category,
            LIMIT)
            
response7 = requests.request("GET", url7, headers = headers)
res7 = response7.json()

#Willowdale West
lat8 = 43.7827364
lng8 = -79.4422593

url8 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat8,
            lng8,
            radius,
            category,
            LIMIT)
            
response8 = requests.request("GET", url8, headers = headers)
res8 = response8.json()

#Woburn
lat9 = 43.7709921
lng9 = -79.2169174

url9 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat9,
            lng9,
            radius,
            category,
            LIMIT)
            
response9 = requests.request("GET", url9, headers = headers)
res9 = response9.json()

#Bayview Village
lat10 = 43.7869473
lng10 = -79.385975

url10 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat8,
            lng8,
            radius,
            category,
            LIMIT)
            
response10 = requests.request("GET", url10, headers = headers)
res10 = response10.json()

#Weston
lat11 = 43.706876
lng11 = -79.5181884

url11 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat11,
            lng11,
            radius,
            category,
            LIMIT)
            
response11 = requests.request("GET", url11, headers = headers)
res11 = response11.json()

In [17]:
search_query = 'Hospital'
categoryId = '4bf58dd8d48988d104941735'

def getNearbyVenues(names, lat1, long1, radius=3000):

    venues_list=[]
    for name, lat, lng in zip(names, lat1, long1):
        
        # create the API request URL
        url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&locale={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query, radius, LIMIT, locale, categoryId)

        
        # make the GET request
        results = requests.get(url1).json()["response"]["venues"]
    
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])

    return(nearby_venues)

In [18]:
all_venues = getNearbyVenues(names=totalDF['Neighbourhood'],
                                   latitudes=totalDF['Latitude'],
                                   longitudes=totalDF['Longitude'])

TypeError: getNearbyVenues() got an unexpected keyword argument 'latitudes'